In [2]:
%cd /content/drive/MyDrive/研究室/研究_M2/src

/content/drive/MyDrive/研究室/研究_M2/src


In [ ]:
import datetime
from pytz import timezone

def Pull_Datetime():
    dates=datetime.datetime.now(timezone('Asia/Tokyo'))
    exept_microsec=dates.strftime("%Y-%m-%d-%H-%M")
    print('[Today : %s]' % str(exept_microsec))

    DATETIME=exept_microsec
    return DATETIME

DATETIME=Pull_Datetime()

[Today : 2021-08-24-23-57]


In [7]:
#DATETIME = '2021-08-24-23-57'

# OUTPUT_PATH =  './Assets/Assets_Output/'
# OUTPUT_DIR=OUTPUT_PATH+ "PersonalEstim_%s/" % DATETIME
# OUTPUT_DIR_RESULTS=OUTPUT_PATH+ "PersonalEstim_%s/results/" % DATETIME

# KEYWORDS_TypeMBTI = '2017-2018'
# KEYWORDS_FileName='2017'

In [ ]:
import os
OUTPUT_PATH =  './Assets/Assets_Output/'
OUTPUT_DIR=OUTPUT_PATH+ "PersonalEstim_%s/" % DATETIME
OUTPUT_DIR_RESULTS=OUTPUT_PATH+ "PersonalEstim_%s/results/" % DATETIME

KEYWORDS_TypeMBTI = '2017-2018'

try:
  os.mkdir(OUTPUT_DIR)
except:
  print(OUTPUT_DIR, '    already exists')
  print()
try:
  os.mkdir(OUTPUT_DIR_RESULTS)
except:
  print(OUTPUT_DIR_RESULTS, '    the directory already exists')
  print()

In [ ]:
 KEYWORDS_FileName='2017'

In [8]:
import yaml
import os # For saving access tokens and for file management when creating and adding to the dataset
import requests # For sending GET requests from the API
import Twitter_API_Key

def Create_url_UserTimeline(list_AuthorID, start_date, end_date, max_results = 10):
    #Change to the endpoint you want to collect data from
    search_url = "https://api.twitter.com/2/users/%s/tweets" % list_AuthorID
    #change params based on the endpoint you are using
    query_params = {'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {},
                    'start_time': start_date,
                    'end_time': end_date
                    }
                    
    return (search_url, query_params)

In [9]:
def Connect_to_endpoint(url, params, next_token = None):
    #prepare params
    params['next_token'] = next_token   #params object received from create_url function
    #pull bearer token
    bearer_token = Twitter_API_Key.Auth_FullAchieve()
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    #request Twtitter
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
        
    return response.json()

In [10]:
import dateutil.parser
import pandas as pd
import tqdm

def Pull_tweets_from_specific_users(list_AuthorID, start_time, end_time, max_results, next_token, OUTPUT_DIR_RESULTS, KEYWORDS_TypeMBTI):
  url = Create_url_UserTimeline(list_AuthorID, start_time, end_time, max_results)
  print("url : ", url)
  json_UserTimeline= Connect_to_endpoint(url[0], url[1], next_token)

  json_UserTimeline
  print(json_UserTimeline)
  list_dict_keys=list(json_UserTimeline.keys())
  if list_dict_keys[0]=='errors':
    return 0
  elif json_UserTimeline['meta']['result_count']==0:
    return 0
  else: 
    list_EachData=json_UserTimeline['data']

  list_csv=[]
  list_csv.clear()
  list_header=['author_id', 'text',  'tweet_id','tweet_time', 'conversation_id','lang']
  for element in list_EachData:
    #print(element['text'])
    tweet_AuthorID=element['author_id']
    tweet_Texts=element['text']
    tweet_id = element['id']
    tweet_time = dateutil.parser.parse(element['created_at'])
    convarsation_id = element['conversation_id']
    tweet_lang = element['lang']

    list_sub_row=[tweet_AuthorID, tweet_Texts, tweet_id, tweet_time, convarsation_id, tweet_lang]
    list_csv.append(list_sub_row)

  df_UserTimeline=pd.DataFrame(list_csv, columns=list_header)
  df_UserTimeline.to_csv('%sTweetsByOneUser_%s_%s.csv' % (OUTPUT_DIR_RESULTS, KEYWORDS_TypeMBTI, df_UserTimeline['author_id'][0]))

  return df_UserTimeline

In [3]:
import pandas as pd

df_2017=pd.read_csv('/content/drive/MyDrive/研究室/研究_M2/src/Assets/Assets_Output/PersonalEstim_2021-08-24-00-36/results/UserListFinal_2017.csv', engine='python')
df_2017=df_2017.dropna(how='any')
df_2017=df_2017.sort_values('tweet_time')
list_AuthorID=df_2017['author_id'].values.tolist()
#list_AuthorID=list(map(int, list_AuthorID))
print(len(list_AuthorID))

df_2017

36244


,Unnamed: 0,author_id,text,tweet_id,tweet_time,lang
36265,466,4.077631e+08,私の性格タイプは “冒険家” です。あなたのタイプは何ですか？ https://t.co/o...,815418374414962689,2017-01-01 04:44:24+00:00,ja
36264,465,1.017366e+09,私の性格タイプは “擁護者” です。あなたのタイプは何ですか？ https://t.co/S...,815456491708633088,2017-01-01 07:15:52+00:00,ja
36263,464,5.890772e+08,私の性格タイプは “エンターテイナー” です。あなたのタイプは何ですか？ https://t...,815469649064128512,2017-01-01 08:08:09+00:00,ja
36262,463,7.817835e+17,私の性格タイプは “建築家” です。あなたのタイプは何ですか？ https://t.co/O...,815523615915446272,2017-01-01 11:42:36+00:00,ja
36261,462,8.792925e+08,私の性格タイプは “論理学者” です。あなたのタイプは何ですか？ https://t.co/...,815531829243453440,2017-01-01 12:15:14+00:00,ja
...,...,...,...,...,...,...
4,4,7.114436e+17,私の性格タイプは “仲介者” です。あなたのタイプは何ですか？ https://t.co/t...,947462646881595395,2017-12-31 13:41:12+00:00,ja
3,3,7.114436e+17,仲介者型の性格 (INFP-A / INFP-T) https://t.co/HtTGyon...,947466597161037824,2017-12-31 13:56:54+00:00,ja
2,2,4.256766e+09,私の性格タイプは “広報運動家” です。あなたのタイプは何ですか？ https://t.co...,947480443783299073,2017-12-31 14:51:55+00:00,ja
1,1,7.081719e+17,私の性格タイプは “仲介者” です。あなたのタイプは何ですか？ https://t.co/y...,947481939396608001,2017-12-31 14:57:52+00:00,ja


In [4]:
print(list_AuthorID)
set_AuthorID=set(list_AuthorID)
print(type(set_AuthorID))
print(len(set_AuthorID))

list_AuthorID = [x for x in set_AuthorID]
print(type(list_AuthorID))

list_AuthorID=list(map(int, list_AuthorID))
print(list_AuthorID)

[407763079.0, 1017366481.0, 589077235.0, 7.817835041996145e+17, 879292549.0, 1073818387.0, 3776470152.0, 81528046.0, 895100538.0, 144518179.0, 63027363.0, 2373660650.0, 91124927.0, 2600052884.0, 15835256.0, 320567011.0, 1016714408.0, 1048316546.0, 196447686.0, 156307662.0, 176472112.0, 300551920.0, 4197861672.0, 168459989.0, 623451336.0, 147244690.0, 101783390.0, 1655804622.0, 102217740.0, 632035287.0, 110581572.0, 2742699577.0, 5323232.0, 631380552.0, 154499212.0, 7.29287430474375e+17, 2530223131.0, 1143991704.0, 2367402313.0, 2417174653.0, 3242318917.0, 2600052884.0, 500412480.0, 1364638332.0, 881553510.0, 3104846480.0, 2162992554.0, 1582298004.0, 1382237816.0, 4463407820.0, 7.29659089299497e+17, 169861814.0, 147173572.0, 7.342386456793129e+17, 4881307401.0, 3771534434.0, 1913684575.0, 2718801900.0, 222850237.0, 531421699.0, 129730985.0, 1348595527.0, 304289212.0, 2327423210.0, 125696490.0, 415545324.0, 127261979.0, 388824439.0, 7.781021945413673e+17, 7.981987250676858e+17, 329988447

In [ ]:
next_token=None
max_results_author=100
start_time = "2017-01-01T00:00:00.000Z"
end_time = "2017-12-31T23:59:59.000Z"

import time

#print(list_AuthorID[6:7])
for idx, element_AuthorID in enumerate(list_AuthorID[20484 + 1:]):
  print(idx + 20484+ 1, "：", element_AuthorID)
  FileName=KEYWORDS_FileName+"_"+str(idx+ 20484 + 1)
  Pull_tweets_from_specific_users(element_AuthorID, start_time, end_time, max_results_author, next_token, OUTPUT_DIR_RESULTS,  FileName)

  time.sleep(1)

In [5]:
print(list_AuthorID.index(720287831331987456))

20460


In [ ]:
list_AuthorID.sort()

In [ ]:
print(list_AuthorID)

[813794, 2538581, 2617501, 3137621, 3318851, 3799191, 3817231, 3826971, 3827441, 3832651, 3846461, 3911141, 3933721, 3967691, 3971351, 3976361, 4027691, 4038981, 4046061, 4053431, 4053801, 4135331, 4173581, 4175081, 4198691, 4466801, 4575811, 4579711, 4819491, 4865291, 4895801, 4945721, 4968651, 5141511, 5221061, 5323232, 5381552, 5394912, 5399732, 5404592, 5435232, 5454252, 5489282, 5516652, 5517152, 5551012, 5551682, 5572342, 5576722, 5579822, 5586262, 5586592, 5588852, 5590582, 5594322, 5601832, 5606642, 5616302, 5628892, 5630532, 5648482, 5651312, 5656572, 5676662, 5680792, 5699952, 5704642, 5720952, 5744412, 5752262, 5761472, 5765552, 5780412, 5785372, 5798412, 5809192, 5809442, 5830352, 5831062, 5831242, 5855722, 5860972, 5868072, 5893592, 5907242, 5926942, 5934682, 5940292, 5978422, 5987532, 6000282, 6010942, 6023862, 6027032, 6064002, 6094672, 6135592, 6137322, 6137682, 6155472, 6177552, 6199672, 6205332, 6234592, 6259132, 6284462, 6305872, 6335552, 6381622, 6385812, 6386802, 6

In [ ]:
with open('/content/drive/MyDrive/sample.txt') as f:
  list_AuthorID=f.readlines()
  for i in range(len(list_AuthorID)):
    list_AuthorID[i]=list_AuthorID[i].rstrip()
  print(list_AuthorID)

['1000405860', '1000793498', '1000856706', '100106333', '1001153851', '1001153851', '1001200189', '1001388404', '1001454445', '1001497244', '100159745', '1001657288', '1001661216', '100169552', '1001777072', '100181366', '100181366', '100216102', '100220967', '100221811', '100224813', '100240066', '100241675', '100246438', '100275798', '100276194', '100284659', '1003112088', '1003302416', '100331880', '1003534698', '100366913', '100385374', '1003948699', '1004103169', '100415776', '100424432', '1004255096', '1004302928', '1004311368', '100439954', '1004421128', '100442673', '100491853', '100493243', '100505173', '1005375080', '1005425216', '100546552', '100551049', '1005586064', '1005658610', '1005879901', '100595489', '100615589', '1006461944', '100665135', '100666656', '1006688292', '10071892', '100733638', '100743653', '100754762', '100759092', '1007898948', '100795885', '1008295765', '1008405062', '1008485912', '1008592706', '100871270', '1008801582', '1008859530', '100888220', '10

In [ ]:
next_token=None
max_results_author=100
start_time = "2017-01-01T00:00:00.000Z"
end_time = "2017-12-31T23:59:59.000Z"

#print(list_AuthorID[6:7])
for idx, element_AuthorID in enumerate(list_AuthorID[0:9]):
  print(idx, "：", element_AuthorID)
  FileName=KEYWORDS_FileName+"_"+str(idx)
  Pull_tweets_from_specific_users(element_AuthorID, start_time, end_time, max_results_author, next_token, OUTPUT_DIR_RESULTS,  FileName)

0 ： 1000405860
url :  ('https://api.twitter.com/2/users/1000405860/tweets', {'max_results': 100, 'expansions': 'author_id,in_reply_to_user_id,geo.place_id', 'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source', 'user.fields': 'id,name,username,created_at,description,public_metrics,verified', 'place.fields': 'full_name,id,country,country_code,geo,name,place_type', 'next_token': {}, 'start_time': '2017-01-01T00:00:00.000Z', 'end_time': '2017-12-31T23:59:59.000Z'})
Endpoint Response Code: 200
{'meta': {'result_count': 0}}
1 ： 1000793498
url :  ('https://api.twitter.com/2/users/1000793498/tweets', {'max_results': 100, 'expansions': 'author_id,in_reply_to_user_id,geo.place_id', 'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source', 'user.fields': 'id,name,username,created_at,description,public_metrics,verif